In [83]:
import folium
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [84]:
dxb_communities=pd.read_csv("Dxb_Comm_LatLon.csv")
dxb_communities.head()

,CNAME_E,COMM_NUM,SHAPE_AREA,SHAPE_LEN,lat,lon
0,AL BADA',333,1.984701e+06,7946.070175,25.225314,55.268768
1,CORNICHE DEIRA,121,8.707842e+05,5165.842443,25.287165,55.317958
2,AL KHWANEEJ SECOND,282,1.269407e+07,14582.351508,25.230697,55.521222
3,AL SAFA FIRST,353,2.430429e+06,6705.662879,25.175690,55.238309
4,NAZWAH,736,1.305883e+07,15567.139691,25.024015,55.646574


In [85]:
print('Dubai has {} communities.'.format(dxb_communities.shape[0]))

Dubai has 226 communities.


In [86]:
#Foursquare credentials

CLIENT_ID = 'CG0J3LO2WFNAGLCF1OMLYIJMZPNS3B53UYOEHHZIETCN3NWY' # your Foursquare ID
CLIENT_SECRET = '2VX0Y1RFXMIUOYDFPQH5H1BY12WBDDTOAU3GH0TR1VIZLRG4' # your Foursquare Secret
VERSION = '20180605'  # Foursquare API version
LIMIT = 100  # A default Foursquare API limit value
HOSPITALID = '4bf58dd8d48988d196941735'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CG0J3LO2WFNAGLCF1OMLYIJMZPNS3B53UYOEHHZIETCN3NWY
CLIENT_SECRET:2VX0Y1RFXMIUOYDFPQH5H1BY12WBDDTOAU3GH0TR1VIZLRG4


# Function to return venues for community


In [87]:
def getNearbyVenues(names, comm_nums, latitudes, longitudes, radius=8000):
    
    venues_list=[]
    for name, comm_num, lat, lng in zip(names, comm_nums, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&categoryId={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            HOSPITALID,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        results
        # return only relevant information for each nearby venue
        venues_list.append([(
            name,
            comm_num,
            lat, 
            lng,
            v['venue']['id'],
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'],) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Community', 
                             'Community_Num',
                             'Community Latitude',
                             'Community Longitude',
                             'Venue ID',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category',]
    
    return(nearby_venues)

In [88]:
# # USE TO TEST FUNCTION getNearbyVenues - ONLY 1 COMMUNITY

# dxb_1community = dxb_communities[dxb_communities['CNAME_E'] == 'CORNICHE DEIRA']


# dxb_1venue = getNearbyVenues(names=dxb_1community['CNAME_E'],
#                              comm_nums=dxb_1community['COMM_NUM'],
#                              latitudes=dxb_1community['lat'],
#                              longitudes=dxb_1community['lon']
#                             )
# dxb_1venue

In [89]:
# # USE THIS CELL TO QUERY FACEBOOK API - ELSE USE NEXT CELL TO LOAD FROM FILE

# dxb_venues = getNearbyVenues(names=dxb_communities['CNAME_E'],
#                              comm_nums=dxb_communities['COMM_NUM'],
#                              latitudes=dxb_communities['lat'],
#                              longitudes=dxb_communities['lon']
#                             )
# dxb_venues

In [90]:
# #USE FOR FILE OUTPUT WHEN DATA IS LOADED FROM FOURSQUARE API

# dxb_venues.to_csv('Dxb_Healthcare_Venues.csv',index=False)

In [91]:
# USE THIS TO LOAD FROM DATA FILE INSTEAD OF FOURSQAURE QUERY

dxb_healthcare_venues=pd.read_csv("Dxb_Healthcare_Venues.csv")
print(dxb_healthcare_venues.shape)
dxb_healthcare_venues.head()



(5475, 9)


,Community,Community_Num,Community Latitude,Community Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,AL BADA',333,25.225314,55.268768,4dd3feece4cd6154148c0509,Iranian Hospital (المستشفى الإيراني),25.230393,55.269144,Hospital
1,AL BADA',333,25.225314,55.268768,4bf1866b52bda593a9a6b1b7,Emirates Hospital (مستشفى الامارات),25.197293,55.237570,Medical Center
2,AL BADA',333,25.225314,55.268768,4bbc29d93de8c9b654b79aad,Medcare Hospital (مستشفى ميدكير),25.183387,55.242802,Hospital
3,AL BADA',333,25.225314,55.268768,4b51a09af964a520245127e3,Dubai Healthcare City مدينة دبي الطبية,25.232799,55.321726,Hospital
4,AL BADA',333,25.225314,55.268768,5e2006c3e7117a0008c8038e,Fakih IVF - Dubai,25.221963,55.263826,Hospital


In [92]:
# map_venues = folium.Map(location=[25, 55], zoom_start=11)

  
# for lat, lng, venue in zip(dxb_venues['Venue Latitude'], dxb_venues['Venue Longitude'], dxb_venues['Venue']):
#     label = folium.Popup(venue, parse_html=True)
#     folium.CircleMarker(
#         [lat, lng],
#         radius=5,
#         popup=label,
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_venues)  
    
# map_venues

In [93]:
# map_communities = folium.Map(location=[25, 55], zoom_start=11)

  
# for lat, lng, comm_name in zip(dxb_communities['lat'], dxb_communities['lon'], dxb_communities['CNAME_E']):
#     label = folium.Popup(comm_name, parse_html=True)
#     folium.Circle(
#         [lat, lng],
#         radius=8000,
#         popup=label,
#         color='blue',
#         fill=True,
#         fill_color='#3186cc',
#         fill_opacity=0.7,
#         parse_html=False).add_to(map_communities)  
    
# map_communities

In [94]:
dxb_healthcare_venues.drop(['Community','Community_Num','Community Latitude','Community Longitude'],axis='columns',inplace=True)
dxb_healthcare_venues.drop_duplicates(subset ="Venue ID", keep = 'first', inplace = True)
dxb_healthcare_venues

,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4dd3feece4cd6154148c0509,Iranian Hospital (المستشفى الإيراني),25.230393,55.269144,Hospital
1,4bf1866b52bda593a9a6b1b7,Emirates Hospital (مستشفى الامارات),25.197293,55.237570,Medical Center
2,4bbc29d93de8c9b654b79aad,Medcare Hospital (مستشفى ميدكير),25.183387,55.242802,Hospital
3,4b51a09af964a520245127e3,Dubai Healthcare City مدينة دبي الطبية,25.232799,55.321726,Hospital
4,5e2006c3e7117a0008c8038e,Fakih IVF - Dubai,25.221963,55.263826,Hospital
...,...,...,...,...,...
2659,4d14ee216c8b548169f2dacc,Cedars Jebel Ali Hospital,25.020539,55.080304,Hospital
2660,51dcfc9a498ec699362742d6,Cedars-Jebel Ali International Hospital,25.015638,55.087743,Hospital
2662,5c8b4758f4b525002cee8be4,Alqudra Dha Clinic,24.842335,55.328217,Hospital
2916,5fe715673850f773449aab68,Neuro Spinal Hospital (Main Branch),25.075684,55.246174,Hospital


### Based on examining the data we identify that the following venues are not actual healthcare facilities :

570545e7498e955deec7a65d	Dubai Healthcare City	25.232924	55.32366	Medical Center (Locality)

5215c35011d2fdcad03d8e5a	Zabeel Veterinary hospital	25.210553	55.288277	Veterinarian (Vet facility)

4d38464dd60c6dcb3b5576bc	NMC	25.27490084	55.34970203	Office (Corp Office of Hospital Company)

4d449e267e2e5481545a628f	Al Shamsi Bldg	25.29202791	55.36881716	Building (Commercial Bldg)

5107bf93e4b09b1bbfa06164	Habtoor Business Tower	25.08562246	55.14167716	Building (Commercial Bldg)



In [95]:
dxb_healthcare_venues.drop(dxb_healthcare_venues[dxb_healthcare_venues['Venue ID'] == '570545e7498e955deec7a65d'].index, inplace = True)
dxb_healthcare_venues.drop(dxb_healthcare_venues[dxb_healthcare_venues['Venue ID'] == '5215c35011d2fdcad03d8e5a'].index, inplace = True)
dxb_healthcare_venues.drop(dxb_healthcare_venues[dxb_healthcare_venues['Venue ID'] == '4d38464dd60c6dcb3b5576bc'].index, inplace = True)
dxb_healthcare_venues.drop(dxb_healthcare_venues[dxb_healthcare_venues['Venue ID'] == '4d449e267e2e5481545a628f'].index, inplace = True)
dxb_healthcare_venues.drop(dxb_healthcare_venues[dxb_healthcare_venues['Venue ID'] == '5107bf93e4b09b1bbfa06164'].index, inplace = True)
dxb_healthcare_venues.reset_index(drop=True, inplace=True)
dxb_healthcare_venues

,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,4dd3feece4cd6154148c0509,Iranian Hospital (المستشفى الإيراني),25.230393,55.269144,Hospital
1,4bf1866b52bda593a9a6b1b7,Emirates Hospital (مستشفى الامارات),25.197293,55.237570,Medical Center
2,4bbc29d93de8c9b654b79aad,Medcare Hospital (مستشفى ميدكير),25.183387,55.242802,Hospital
3,4b51a09af964a520245127e3,Dubai Healthcare City مدينة دبي الطبية,25.232799,55.321726,Hospital
4,5e2006c3e7117a0008c8038e,Fakih IVF - Dubai,25.221963,55.263826,Hospital
...,...,...,...,...,...
144,4d14ee216c8b548169f2dacc,Cedars Jebel Ali Hospital,25.020539,55.080304,Hospital
145,51dcfc9a498ec699362742d6,Cedars-Jebel Ali International Hospital,25.015638,55.087743,Hospital
146,5c8b4758f4b525002cee8be4,Alqudra Dha Clinic,24.842335,55.328217,Hospital
147,5fe715673850f773449aab68,Neuro Spinal Hospital (Main Branch),25.075684,55.246174,Hospital


In [96]:
dxb_healthcare_venues.to_csv('Dxb_Healthcare_Venues_Cleaned.csv',index=False)